In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DIR_NAME = os.getcwd()
FNAME = 'Feats.csv'
YEAR_BEG = 13
YEAR_END = 19
NUM_GAMES = 38
MIN_GAMES = 5
WINDOW_LEN = 3


We store the data as a list of DataFrames, though it would be better to utilize Panda's hierarchical indexing functionality, as we do with 
the final feature matrix.

In [3]:
def read_data(beg, end):
    '''
    Stores data for all seasons between beg and end in separate DataFrames.
    
    Parameters:
    beg (int): int of the season to start at (e.g. 12)
    end (int): int of the season to end at
    
    Returns:
    datasets (list): list of DataFrames each containing the data for a particular season
    '''
    years = [str(i) + str(i+1) for i in range(beg,end)]
    datasets = []
    for suffix in years:
        csv = DIR_NAME + '/data/match_stats' + suffix + '.csv' 
        dat = pd.read_csv(csv)
        datasets.append(dat)  
    return datasets

datasets = read_data(YEAR_BEG, YEAR_END)
for data in datasets:
    print(data)

    Div      Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0    E0  17/08/13      Arsenal  Aston Villa     1     3   A     1     1   D   
1    E0  17/08/13    Liverpool        Stoke     1     0   H     1     0   H   
2    E0  17/08/13      Norwich      Everton     2     2   D     0     0   D   
3    E0  17/08/13   Sunderland       Fulham     0     1   A     0     0   D   
4    E0  17/08/13      Swansea   Man United     1     4   A     0     2   A   
..   ..       ...          ...          ...   ...   ...  ..   ...   ...  ..   
375  E0  11/05/14      Norwich      Arsenal     0     2   A     0     0   D   
376  E0  11/05/14  Southampton   Man United     1     1   D     1     0   H   
377  E0  11/05/14   Sunderland      Swansea     1     3   A     0     2   A   
378  E0  11/05/14    Tottenham  Aston Villa     3     0   H     3     0   H   
379  E0  11/05/14    West Brom        Stoke     1     2   A     0     1   A   

     ... BbAv<2.5  BbAH  BbAHh  BbMxAHH  BbAvAHH  B

To help create our features we make indicator columns for the results as well as a column indicating the week during which the match took place.

In [4]:
for i, data in enumerate(datasets):
    if i in [2,4,5]:
        format = '%d/%m/%Y'
    else:
        format = '%d/%m/%y'
    data['HWIN'] = data['FTHG'] > data['FTAG']
    data['DRAW'] = data['FTR'] == 'D'
    data['Week'] = pd.to_datetime(data['Date'], format=format).apply(lambda x: x.isocalendar()[1])
    
    

In [5]:
def compute_homewin_rate(df):
    '''
    Computes the overall win percentage of teams playing at home using all the matches up to the previous week.
    Parameters:
    df (DataFrame): Contains the week during which a match was played and an indicator for a home team win. One observation per match.
    Returns:
    hwin_rate (Series): The overall home team win percentage up to the given week. Note it contains an entry for each match, so all 
                        matches with the same week will have the same percentage

    '''
    df = df.groupby('Week', sort=False).agg(['sum','count']).cumsum()
    df.columns = df.columns.get_level_values(1)
    hwin_rate = df['sum'] / df['count']
    hwin_rate.name = 'HwinFreq'
    return hwin_rate.shift(1).dropna()


In [6]:
datasets = [data.merge(compute_homewin_rate(data.loc[:,('Week', 'HWIN')]), how='left', on='Week') for data in datasets]

In [7]:
def compute_team_str(df, name, home):
    '''
    Computes the strength of team, which is defined to be the difference between its win rate and loss rate. Since
    the performace of a team can vary greatly depending on whether or not it is playing at home, two strength values 
    are computed for each team: one using home games only, the other away games. Values that would have been computed 
    using a sample of size less than MIN_PERIODS, a constant that has been set to 5, are treated as not representative 
    of the true value, so they are discarded.
    Parameters:
    df (DataFrame): contains following fields: if home is True HomeTeam, otherwise AwayTeam, HWIN (indicator for home team win),
                    DRAW (indicator for a draw). One obeservation per match.
    name (String): the name of the team
    home (Bool): indicates whether it is the home strength or the away strength being computed
    Returns:
    team_str (Series): the strength of a team using all its data up to the last match it played  
    '''
    if home:
        col = 'HomeTeam'
        sgn = 1
    else:
        col = 'AwayTeam'
        sgn = -1
    df = df.loc[df[col] == name]
    a = df['HWIN'].expanding(min_periods=MIN_GAMES).mean()
    b = (~df['HWIN'] & ~df['DRAW']).expanding(min_periods=MIN_GAMES).mean() 
    team_str = sgn * (a - b)
    team_str.name = col + 'Str'
    #print(home_team_str)
    return team_str.shift(1).dropna()
    

In [8]:
for col in ['HomeTeam', 'AwayTeam']:
    if col == 'HomeTeam':
        home = True
    else:
        home = False
    datasets = [data.merge(pd.concat([compute_team_str(data.loc[:,[col]+['HWIN','DRAW']], name, home
                                                      ) for name in pd.unique(data[col].values)]), 
                           how='left', left_index=True, right_index=True) for data in datasets]

In [9]:
def compute_form(df, name):
    '''
    Computes the recent form of a team, which is defined to be the sum of their goal difference over the past WINDOW_LEN (set at 3) 
    number of games.
    Parameters:
    df (DataFrame): contains the following information: team names and goals scored by each team. One observation per match.
    name (String):
    Returns:
    gd (Series): the recent form of a team over the last WINDOW_LEN (3) matches it has played
    '''
    df = df[(df['HomeTeam']==name) | (df['AwayTeam']==name)].loc[:,('HomeTeam', 'FTHG', 'FTAG')]
    gd = np.where(np.vstack((df['HomeTeam']==name,df['HomeTeam']==name)),
                  np.vstack((np.ones(NUM_GAMES),df['FTHG']-df['FTAG'])),np.vstack((np.zeros(NUM_GAMES),df['FTAG']-df['FTHG'])))
    gd = pd.DataFrame(gd.T, index=df.index, columns=['Home','GD'])
    gd['GD'] = gd['GD'].rolling(window=WINDOW_LEN).sum().shift(1)
    return gd.dropna()

    

In [10]:
for data in datasets:
    temp = pd.concat([compute_form(data.loc[:,['HomeTeam', 'AwayTeam','FTHG','FTAG']], name) for name in pd.unique(data['HomeTeam'].values)])
    temp['Home'] = temp['Home'].astype('boolean')
    data['HomeTeamForm'] = temp[temp['Home']]['GD']
    data['AwayTeamForm'] = temp[~temp['Home']]['GD']
    

In [11]:
full_df = pd.concat(datasets, keys=[str(i) + '-' + str(i+1) for i in range(YEAR_BEG,YEAR_END)]).loc[:,('HwinFreq','HomeTeamStr','AwayTeamStr','HomeTeamForm','AwayTeamForm','FTR',
                                    'B365H', 'B365D', 'B365A')].dropna()

In [12]:
full_df

HwinFreq  HomeTeamStr  AwayTeamStr  HomeTeamForm  AwayTeamForm FTR  \
13-14 100  0.460000    -0.600000    -0.200000          -4.0          -2.0   H   
      101  0.460000     1.000000    -0.000000           2.0          -1.0   D   
      102  0.460000    -0.600000     0.200000          -7.0           3.0   D   
      103  0.460000     0.600000    -0.200000           1.0          -1.0   H   
      104  0.460000    -0.200000    -0.000000         -10.0          -2.0   H   
...             ...          ...          ...           ...           ...  ..   
18-19 375  0.482385     0.888889    -0.055556           8.0           4.0   H   
      376  0.482385     0.444444    -0.555556          -2.0          -4.0   A   
      377  0.482385    -0.055556    -0.722222          -3.0          -6.0   D   
      378  0.482385     0.388889    -0.222222          -1.0           6.0   D   
      379  0.482385     0.055556    -0.277778          -4.0           4.0   A   

           B365H  B365D  B365A  
13-14 100   2.00   3.50   4.20  
      101   1.29   6.00  12.00  
      102   6.50   4.00   1.62  
      103   1.25   6.50  15.00  
      104   2.63   3.30   2.90  
...          ...    ...    ...  
18-19 375   1.30   6.00  11.00  
      376   1.28   6.50  11.00  
      377   1.44   4.75   8.50  
      378   2.20   3.50   3.50  
      379   2.25   3.75   3.20  

[1674 rows x 9 columns]

In [13]:
full_df.to_csv(DIR_NAME + '/Data/' + FNAME)